In [33]:
pip install scikit-multilearn

  Obtaining dependency information for scikit-multilearn from https://files.pythonhosted.org/packages/bb/1f/e6ff649c72a1cdf2c7a1d31eb21705110ce1c5d3e7e26b2cc300e1637272/scikit_multilearn-0.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 834.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from skmultilearn.adapt import MLkNN 
from sklearn.metrics import hamming_loss, accuracy_score 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
# DATA Exploration

In [108]:
# import data
df= pd.read_csv('Symptom2Disease.csv')
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1200 non-null   int64 
 1   label       1200 non-null   object
 2   text        1200 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.3+ KB


In [110]:
df.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [7]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [112]:
# Check for data classes
df['label'].nunique()

24

In [113]:
unique_values = df['text'].unique()
unique_count = df['text'].nunique()

#print("Unique values:", unique_values)
print("Count of unique values:", unique_count)

Count of unique values: 1153


In [114]:
a = df['label'].unique()
print(a)
print(type(a))

['Psoriasis' 'Varicose Veins' 'Typhoid' 'Chicken pox' 'Impetigo' 'Dengue'
 'Fungal infection' 'Common Cold' 'Pneumonia' 'Dimorphic Hemorrhoids'
 'Arthritis' 'Acne' 'Bronchial Asthma' 'Hypertension' 'Migraine'
 'Cervical spondylosis' 'Jaundice' 'Malaria' 'urinary tract infection'
 'allergy' 'gastroesophageal reflux disease' 'drug reaction'
 'peptic ulcer disease' 'diabetes']
<class 'numpy.ndarray'>


In [115]:
# sort target data
target=['Psoriasis', 'Varicose Veins', 'Typhoid', 'Chicken pox',
       'Impetigo', 'Dengue', 'Fungal infection', 'Common Cold',
       'Pneumonia', 'Dimorphic Hemorrhoids', 'Arthritis', 'Acne',
       'Bronchial Asthma', 'Hypertension', 'Migraine',
       'Cervical spondylosis', 'Jaundice', 'Malaria',
       'urinary tract infection', 'allergy',
       'gastroesophageal reflux disease', 'drug reaction',
       'peptic ulcer disease', 'diabetes']
real_target= sorted(target)

In [116]:
real_target

['Acne',
 'Arthritis',
 'Bronchial Asthma',
 'Cervical spondylosis',
 'Chicken pox',
 'Common Cold',
 'Dengue',
 'Dimorphic Hemorrhoids',
 'Fungal infection',
 'Hypertension',
 'Impetigo',
 'Jaundice',
 'Malaria',
 'Migraine',
 'Pneumonia',
 'Psoriasis',
 'Typhoid',
 'Varicose Veins',
 'allergy',
 'diabetes',
 'drug reaction',
 'gastroesophageal reflux disease',
 'peptic ulcer disease',
 'urinary tract infection']

In [117]:
df.shape

(1200, 2)

In [118]:
df.duplicated().sum()

47

In [119]:
df[df.duplicated]

,label,text
163,Chicken pox,I'm feeling fatigued and have no energy. I can...
387,Common Cold,I've been quite exhausted and ill. My throat h...
430,Pneumonia,"I have a really high fever, and I have problem..."
433,Pneumonia,I'm having a hard time breathing and I feel re...
438,Pneumonia,"Lately I've been experiencing chills, fatigue,..."
469,Dimorphic Hemorrhoids,I've been constipated and it's really hard to ...
470,Dimorphic Hemorrhoids,"Since I've been constipated, using the restroo..."
471,Dimorphic Hemorrhoids,I've been constipated and it's really hard to ...
487,Dimorphic Hemorrhoids,Lately I've been experiencing constipation and...
489,Dimorphic Hemorrhoids,I've recently been suffering from constipation...


In [120]:
df.drop_duplicates(inplace= True)

In [121]:
df.shape

(1153, 2)

In [122]:
df['label'].value_counts()

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
allergy                            50
urinary tract infection            50
Hypertension                       50
diabetes                           50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Typhoid                            50
Common Cold                        49
Cervical spondylosis               49
Chicken pox                        49
Bronchial Asthma                   49
gastroesophageal reflux disease    48
Pneumonia                          47
Migraine                           47
Arthritis                          46
Acne                               46
Malaria                            44
Dimorphic Hemorrhoids              41
Jaundice                           38
Name: count, dtype: int64

In [123]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [124]:
df.tail()

,label,text
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


In [125]:
df.isnull().sum() # to check null value

label    0
text     0
dtype: int64

In [59]:
#df.drop('label',axis=1, inplace=True)

In [126]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [127]:
df.tail()

,label,text
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


#### One hot Encoding

In [128]:
#Extract categorical columns from the dataframe
categorical_columns = ['label']
#Initialize OneHotEncoder
encoder = OneHotEncoder()

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[['label']])


In [129]:
# Convert sparse matrix to a dense array
one_hot_encoded_array = one_hot_encoded.toarray()
# # Create new column names based on the categories
column_names = encoder.get_feature_names_out(['label'])
# # Create a DataFrame from the one-hot encoded array
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_array, columns=column_names)



In [130]:
one_hot_encoded_df

,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,label_Hypertension,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1149,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [131]:
df['text']

0       I have been experiencing a skin rash on my arm...
1       My skin has been peeling, especially on my kne...
2       I have been experiencing joint pain in my fing...
3       There is a silver like dusting on my skin, esp...
4       My nails have small dents or pits in them, and...
                              ...                        
1195    I'm shaking and trembling all over. I've lost ...
1196    Particularly in the crevices of my skin, I hav...
1197    I regularly experience these intense urges and...
1198    I have trouble breathing, especially outside. ...
1199    I constantly sneeze and have a dry cough. My i...
Name: text, Length: 1153, dtype: object

In [72]:
df['text'].isnull().sum() # to check null value

0

In [132]:
text_df = df.drop('label', axis=1)

In [133]:
text_df

,text
0,I have been experiencing a skin rash on my arm...
1,"My skin has been peeling, especially on my kne..."
2,I have been experiencing joint pain in my fing...
3,"There is a silver like dusting on my skin, esp..."
4,"My nails have small dents or pits in them, and..."
...,...
1195,I'm shaking and trembling all over. I've lost ...
1196,"Particularly in the crevices of my skin, I hav..."
1197,I regularly experience these intense urges and...
1198,"I have trouble breathing, especially outside. ..."


In [134]:
text_df.isnull().sum() # to check null value

text    0
dtype: int64

In [135]:
# Concatenate the one-hot encoded DataFrame with the original DataFrame
df_with_one_hot = pd.concat([text_df, one_hot_encoded_df], axis=1)


In [136]:

df_with_one_hot

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
0,I have been experiencing a skin rash on my arm...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"My skin has been peeling, especially on my kne...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,I have been experiencing joint pain in my fing...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"There is a silver like dusting on my skin, esp...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"My nails have small dents or pits in them, and...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
873,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
894,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1048,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [137]:
df_with_one_hot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 1049
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   text                                   1153 non-null   object 
 1   label_Acne                             1153 non-null   float64
 2   label_Arthritis                        1153 non-null   float64
 3   label_Bronchial Asthma                 1153 non-null   float64
 4   label_Cervical spondylosis             1153 non-null   float64
 5   label_Chicken pox                      1153 non-null   float64
 6   label_Common Cold                      1153 non-null   float64
 7   label_Dengue                           1153 non-null   float64
 8   label_Dimorphic Hemorrhoids            1153 non-null   float64
 9   label_Fungal infection                 1153 non-null   float64
 10  label_Hypertension                     1153 non-null   float64
 11  label_Imp

In [138]:
df_with_one_hot['text'].isnull().sum() # to check null value

47

In [139]:
df_with_one_hot.isnull().sum() # to check null value


text                                     47
label_Acne                               47
label_Arthritis                          47
label_Bronchial Asthma                   47
label_Cervical spondylosis               47
label_Chicken pox                        47
label_Common Cold                        47
label_Dengue                             47
label_Dimorphic Hemorrhoids              47
label_Fungal infection                   47
label_Hypertension                       47
label_Impetigo                           47
label_Jaundice                           47
label_Malaria                            47
label_Migraine                           47
label_Pneumonia                          47
label_Psoriasis                          47
label_Typhoid                            47
label_Varicose Veins                     47
label_allergy                            47
label_diabetes                           47
label_drug reaction                      47
label_gastroesophageal reflux di

In [140]:
df_cleaned = df_with_one_hot.dropna()


In [141]:
df_cleaned['text'].isnull().sum() # to check null value

0

In [142]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1106 entries, 0 to 1152
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   text                                   1106 non-null   object 
 1   label_Acne                             1106 non-null   float64
 2   label_Arthritis                        1106 non-null   float64
 3   label_Bronchial Asthma                 1106 non-null   float64
 4   label_Cervical spondylosis             1106 non-null   float64
 5   label_Chicken pox                      1106 non-null   float64
 6   label_Common Cold                      1106 non-null   float64
 7   label_Dengue                           1106 non-null   float64
 8   label_Dimorphic Hemorrhoids            1106 non-null   float64
 9   label_Fungal infection                 1106 non-null   float64
 10  label_Hypertension                     1106 non-null   float64
 11  label_Imp

In [143]:
df_cleaned.head()

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
0,I have been experiencing a skin rash on my arm...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"My skin has been peeling, especially on my kne...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,I have been experiencing joint pain in my fing...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"There is a silver like dusting on my skin, esp...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"My nails have small dents or pits in them, and...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [144]:
df_cleaned.tail()

,text,label_Acne,label_Arthritis,label_Bronchial Asthma,label_Cervical spondylosis,label_Chicken pox,label_Common Cold,label_Dengue,label_Dimorphic Hemorrhoids,label_Fungal infection,...,label_Pneumonia,label_Psoriasis,label_Typhoid,label_Varicose Veins,label_allergy,label_diabetes,label_drug reaction,label_gastroesophageal reflux disease,label_peptic ulcer disease,label_urinary tract infection
1148,"My mouth is filled with a sour, acidic flavour...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1149,I unintentionally lose weight and find it chal...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1150,I have increased thirst and frequent urination...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1151,I have blurred vision and it only seems to be ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1152,I have a dry mouth and throat. I also have bee...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Formatdf_cleaned Training

In [152]:
df_cleaned.shape

(1106, 25)

In [145]:
X = df_cleaned["text"] 
y = np.asarray(df_cleaned[df_cleaned.columns[1:]]) 

In [146]:
X

0       I have been experiencing a skin rash on my arm...
1       My skin has been peeling, especially on my kne...
2       I have been experiencing joint pain in my fing...
3       There is a silver like dusting on my skin, esp...
4       My nails have small dents or pits in them, and...
                              ...                        
1148    My mouth is filled with a sour, acidic flavour...
1149    I unintentionally lose weight and find it chal...
1150    I have increased thirst and frequent urination...
1151    I have blurred vision and it only seems to be ...
1152    I have a dry mouth and throat. I also have bee...
Name: text, Length: 1106, dtype: object

In [166]:
len(y[0])

24

In [148]:
# initializing TfidfVectorizer  
vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85) 
# fitting the tf-idf on the given data 
vetorizar.fit(X) 
  
# splitting the data to training and testing data set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42) 
  
# transforming the data 
X_train_tfidf = vetorizar.transform(X_train) 
X_test_tfidf = vetorizar.transform(X_test) 

In [167]:
X_train_tfidf[0]

<1x1494 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [157]:
# using Multi-label kNN classifier 
mlknn_classifier = MLkNN(k=3)
mlknn_classifier

MLkNN(k=3)

In [158]:
type(X_train_tfidf)

scipy.sparse._csr.csr_matrix

In [159]:
X_train_tfidf

<995x1494 sparse matrix of type '<class 'numpy.float64'>'
	with 23007 stored elements in Compressed Sparse Row format>

In [160]:
type(y_train)

numpy.ndarray

In [168]:
mlknn_classifier.fit(X_train,y_train.reshape(-1, 1))

TypeError: NearestNeighbors.__init__() takes 1 positional argument but 2 were given